In [6]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

from pathlib import Path

from src.processing.splitting import time_based_split
from src.processing.scaling import scale_features
from src.processing.sequence_creator import create_sequences
from src.training.lstm_training import train_lstm
from config import config


df_btc = pd.read_csv(
    config.DATA_DIR / "processed" / "crypto_prices" / "btc.csv",
    parse_dates=["date"],
    index_col="date",
)
df_eth = pd.read_csv(
    config.DATA_DIR / "processed" / "crypto_prices" / "eth.csv",
    parse_dates=["date"],
    index_col="date",
)

## Split


In [7]:
X_train_btc, y_train_btc, X_val_btc, y_val_btc, X_test_btc, y_test_btc = (
    time_based_split(
        df_btc,
        targets=["logPriceChange"],
        test_months=12,
        val_months=3,
        lags=30,
    )
)

X_train_eth, y_train_eth, X_val_eth, y_val_eth, X_test_eth, y_test_eth = (
    time_based_split(
        df_eth,
        targets="logPriceChange",
        test_months=12,
        val_months=3,
        lags=30,
    )
)

In [29]:
print(y_test_btc.tail())
print(y_test_btc.head())

            logPriceChange
date                      
2022-12-16       -0.042190
2022-12-17        0.008828
2022-12-18       -0.002212
2022-12-19       -0.019176
2022-12-20        0.027989
            logPriceChange
date                      
2021-11-21       -0.016326
2021-11-22       -0.042455
2021-11-23        0.022481
2021-11-24       -0.022639
2021-11-25        0.017512


## Scale


In [8]:
X_train_scaled_btc, X_val_scaled_btc, X_test_scaled_btc, scaler_btc = scale_features(
    X_train_btc, X_val_btc, X_test_btc
)

X_train_scaled_eth, X_val_scaled_eth, X_test_scaled_eth, scaler_eth = scale_features(
    X_train_eth, X_val_eth, X_test_eth
)

## Sequence Creation


In [9]:
X_train_seq_btc, y_train_seq_btc = create_sequences(
    X_train_scaled_btc, y_train_btc, seq_length=30
)
X_val_seq_btc, y_val_seq_btc = create_sequences(
    X_val_scaled_btc, y_val_btc, seq_length=30
)
X_test_seq_btc, y_test_seq_btc = create_sequences(
    X_test_scaled_btc, y_test_btc, seq_length=30
)

print(f"X_train_seq_btc shape: {X_train_seq_btc.shape}")
print(f"y_train_seq_btc shape: {y_train_seq_btc.shape}")

X_train_seq_eth, y_train_seq_eth = create_sequences(
    X_train_scaled_eth, y_train_eth, seq_length=30
)
X_val_seq_eth, y_val_seq_eth = create_sequences(
    X_val_scaled_eth, y_val_eth, seq_length=30
)
X_test_seq_eth, y_test_seq_eth = create_sequences(
    X_test_scaled_eth, y_test_eth, seq_length=30
)

X_train_seq_btc shape: (1663, 30, 72)
y_train_seq_btc shape: (1663, 1)


# BTC


### Model Tuning

In [ ]:
from src.training.hyperparameters_tuning import tune_hyperparameters

study_lstm_btc = tune_hyperparameters(
    X_train_seq_btc,
    y_train_seq_btc,
    X_val_seq_btc,
    y_val_seq_btc,
    train_lstm,
    n_trials=50,
)

## Model Training

In [11]:
# best parameters from the Optuna study
    """
    Params:
    num_lstm_layers: 3
    lstm_units_1: 256
    lstm_units_2: 32
    lstm_units_3: 64
    num_dense_layers: 1
    dense_units_1: 224
    dense_units_2: 224
    dense_units_3: 64
    dropout_rate: 0.30000000000000004
    learning_rate: 0.00035070707045515984
    batch_size: 64
    """
best_params = study_lstm_btc.best_params

# Format the parameters correctly
formatted_params = {
    "lstm_units": [
        best_params[f"lstm_units_{i+1}"] for i in range(best_params["num_lstm_layers"])
    ],
    "dense_units": [
        best_params[f"dense_units_{i+1}"]
        for i in range(best_params["num_dense_layers"])
    ],
    "dropout_rate": best_params["dropout_rate"],
    "learning_rate": best_params["learning_rate"],
    "batch_size": best_params["batch_size"],
    "epochs": 300,
}

# Train the model with the formatted parameters
model_btc, history_btc = train_lstm(
    X_train_seq_btc,
    y_train_seq_btc,
    X_val_seq_btc,
    y_val_seq_btc,
    params=formatted_params,
)

Epoch 1/300
26/26 ━━━━━━━━━━━━━━━━━━━━ 4s 68ms/step - loss: 0.0044 - mae: 0.0498 - val_loss: 0.0017 - val_mae: 0.0326
Epoch 2/300
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 59ms/step - loss: 0.0031 - mae: 0.0409 - val_loss: 0.0012 - val_mae: 0.0274
Epoch 3/300
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step - loss: 0.0025 - mae: 0.0363 - val_loss: 0.0011 - val_mae: 0.0259
Epoch 4/300
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 59ms/step - loss: 0.0024 - mae: 0.0354 - val_loss: 0.0012 - val_mae: 0.0273
Epoch 5/300
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 63ms/step - loss: 0.0023 - mae: 0.0344 - val_loss: 0.0012 - val_mae: 0.0271
Epoch 6/300
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 60ms/step - loss: 0.0024 - mae: 0.0351 - val_loss: 0.0010 - val_mae: 0.0256
Epoch 7/300
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 60ms/step - loss: 0.0021 - mae: 0.0326 - val_loss: 0.0011 - val_mae: 0.0265
Epoch 8/300
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 61ms/step - loss: 0.0021 - mae: 0.0328 - val_loss: 0.0011 - val_mae: 0.0263
Epoch 9/300
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 59ms/step - lo

In [30]:
# save weights
model_btc.save_weights(
    config.DATA_DIR
    / "weights"
    / "log_price_change"
    / "log_price_change_weights_btc.weights.h5"
)

#  generate predictions
y_pred_btc = model_btc.predict(X_test_seq_btc).flatten()

# create new df and save it to csv
predictions_btc = pd.DataFrame(
    {
        "date": y_test_btc.index[30:],
        "predicted_log_price_change": y_pred_btc,
    }
)


predictions_btc.to_csv(
    config.DATA_DIR / "predictions" / "log_price_change_predictions_btc.csv",
    index=False,
)

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step


# ETH

### Model Tuning

In [32]:
study_lstm_eth = tune_hyperparameters(
    X_train_seq_eth,
    y_train_seq_eth,
    X_val_seq_eth,
    y_val_seq_eth,
    train_lstm,
    n_trials=50,
)

[I 2025-01-28 15:07:31,667] A new study created in memory with name: no-name-00c7df5e-42d7-48ce-b7d8-0ace39c5e9f8


Epoch 1/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 4s 76ms/step - loss: 0.0093 - mae: 0.0718 - val_loss: 0.0020 - val_mae: 0.0338
Epoch 2/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 75ms/step - loss: 0.0070 - mae: 0.0604 - val_loss: 0.0018 - val_mae: 0.0337
Epoch 3/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 76ms/step - loss: 0.0066 - mae: 0.0583 - val_loss: 0.0018 - val_mae: 0.0339
Epoch 4/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 67ms/step - loss: 0.0062 - mae: 0.0577 - val_loss: 0.0018 - val_mae: 0.0340
Epoch 5/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 69ms/step - loss: 0.0058 - mae: 0.0533 - val_loss: 0.0018 - val_mae: 0.0346
Epoch 6/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 71ms/step - loss: 0.0057 - mae: 0.0551 - val_loss: 0.0018 - val_mae: 0.0342
Epoch 7/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 72ms/step - loss: 0.0055 - mae: 0.0526 - val_loss: 0.0017 - val_mae: 0.0340
Epoch 8/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 71ms/step - loss: 0.0055 - mae: 0.0534 - val_loss: 0.0017 - val_mae: 0.0335
Epoch 9/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 74ms/step - lo

[I 2025-01-28 15:10:40,489] Trial 0 finished with value: 0.0015064141480252147 and parameters: {'num_lstm_layers': 3, 'lstm_units_1': 128, 'lstm_units_2': 192, 'lstm_units_3': 32, 'num_dense_layers': 3, 'dense_units_1': 96, 'dense_units_2': 256, 'dense_units_3': 160, 'dropout_rate': 0.30000000000000004, 'learning_rate': 1.6619728688097148e-05, 'batch_size': 64}. Best is trial 0 with value: 0.0015064141480252147.


Epoch 1/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - loss: 0.0884 - mae: 0.2237 - val_loss: 0.1234 - val_mae: 0.3020
Epoch 2/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.1632 - mae: 0.3008 - val_loss: 0.1298 - val_mae: 0.3167
Epoch 3/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - loss: 0.3187 - mae: 0.4000 - val_loss: 0.0350 - val_mae: 0.1510
Epoch 4/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 0.0443 - mae: 0.1515 - val_loss: 0.0084 - val_mae: 0.0760
Epoch 5/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 0.0437 - mae: 0.1521 - val_loss: 0.0486 - val_mae: 0.1922
Epoch 6/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 0.0296 - mae: 0.1212 - val_loss: 0.0180 - val_mae: 0.1157
Epoch 7/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 0.0102 - mae: 0.0793 - val_loss: 0.0101 - val_mae: 0.0856
Epoch 8/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 0.0100 - mae: 0.0775 - val_loss: 0.0056 - val_mae: 0.0557
Epoch 9/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - lo

[I 2025-01-28 15:11:46,128] Trial 1 finished with value: 0.003047868376597762 and parameters: {'num_lstm_layers': 1, 'lstm_units_1': 192, 'lstm_units_2': 160, 'lstm_units_3': 128, 'num_dense_layers': 0, 'dense_units_1': 224, 'dense_units_2': 256, 'dense_units_3': 32, 'dropout_rate': 0.1, 'learning_rate': 0.0014706872947752555, 'batch_size': 64}. Best is trial 0 with value: 0.0015064141480252147.


...

Epoch 1/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 3s 34ms/step - loss: 0.0071 - mae: 0.0632 - val_loss: 0.0016 - val_mae: 0.0331
Epoch 2/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - loss: 0.0047 - mae: 0.0477 - val_loss: 0.0015 - val_mae: 0.0308
Epoch 3/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - loss: 0.0044 - mae: 0.0460 - val_loss: 0.0015 - val_mae: 0.0302
Epoch 4/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - loss: 0.0043 - mae: 0.0458 - val_loss: 0.0015 - val_mae: 0.0304
Epoch 5/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - loss: 0.0042 - mae: 0.0442 - val_loss: 0.0014 - val_mae: 0.0300
Epoch 6/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - loss: 0.0042 - mae: 0.0441 - val_loss: 0.0014 - val_mae: 0.0304
Epoch 7/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - loss: 0.0041 - mae: 0.0437 - val_loss: 0.0015 - val_mae: 0.0309
Epoch 8/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - loss: 0.0039 - mae: 0.0430 - val_loss: 0.0013 - val_mae: 0.0290
Epoch 9/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step - lo

[I 2025-01-28 17:11:46,841] Trial 48 finished with value: 0.0008420322556048632 and parameters: {'num_lstm_layers': 2, 'lstm_units_1': 128, 'lstm_units_2': 96, 'lstm_units_3': 64, 'num_dense_layers': 3, 'dense_units_1': 64, 'dense_units_2': 96, 'dense_units_3': 224, 'dropout_rate': 0.2, 'learning_rate': 0.0006249870035738854, 'batch_size': 32}. Best is trial 11 with value: 0.0007401785114780068.


Epoch 1/100
104/104 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - loss: 0.0585 - mae: 0.1847 - val_loss: 0.0190 - val_mae: 0.1130
Epoch 2/100
104/104 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 0.0168 - mae: 0.0986 - val_loss: 0.0040 - val_mae: 0.0529
Epoch 3/100
104/104 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 0.0065 - mae: 0.0582 - val_loss: 0.0028 - val_mae: 0.0418
Epoch 4/100
104/104 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 0.0052 - mae: 0.0498 - val_loss: 0.0019 - val_mae: 0.0341
Epoch 5/100
104/104 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 0.0045 - mae: 0.0467 - val_loss: 0.0015 - val_mae: 0.0308
Epoch 6/100
104/104 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 0.0044 - mae: 0.0443 - val_loss: 0.0012 - val_mae: 0.0281
Epoch 7/100
104/104 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - loss: 0.0042 - mae: 0.0439 - val_loss: 0.0013 - val_mae: 0.0278
Epoch 8/100
104/104 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 0.0041 - mae: 0.0427 - val_loss: 0.0012 - val_mae: 0.0276
Epoch 9/100
104/104 ━━━━━━━━━━━━━━━━━━━━

[I 2025-01-28 17:13:35,768] Trial 49 finished with value: 0.0009147262899205089 and parameters: {'num_lstm_layers': 1, 'lstm_units_1': 160, 'lstm_units_2': 64, 'lstm_units_3': 128, 'num_dense_layers': 1, 'dense_units_1': 96, 'dense_units_2': 32, 'dense_units_3': 192, 'dropout_rate': 0.30000000000000004, 'learning_rate': 0.0010447732386603937, 'batch_size': 16}. Best is trial 11 with value: 0.0007401785114780068.


Best trial:
  Value (val_loss): 0.0007401785114780068
  Params:
    num_lstm_layers: 2
    lstm_units_1: 160
    lstm_units_2: 32
    lstm_units_3: 32
    num_dense_layers: 1
    dense_units_1: 96
    dense_units_2: 64
    dense_units_3: 192
    dropout_rate: 0.2
    learning_rate: 0.0005577190382479473
    batch_size: 32


## Model Training

In [34]:
# best parameters from the Optuna study
"""
    Params:
    num_lstm_layers: 2
    lstm_units_1: 160
    lstm_units_2: 32
    lstm_units_3: 32
    num_dense_layers: 1
    dense_units_1: 96
    dense_units_2: 64
    dense_units_3: 192
    dropout_rate: 0.2
    learning_rate: 0.0005577190382479473
    batch_size: 32
"""
best_params = study_lstm_eth.best_params

# Format the parameters correctly
formatted_params = {
    "lstm_units": [
        best_params[f"lstm_units_{i+1}"] for i in range(best_params["num_lstm_layers"])
    ],
    "dense_units": [
        best_params[f"dense_units_{i+1}"]
        for i in range(best_params["num_dense_layers"])
    ],
    "dropout_rate": best_params["dropout_rate"],
    "learning_rate": best_params["learning_rate"],
    "batch_size": best_params["batch_size"],
    "epochs": 300,
}

# Train the model with the formatted parameters
model_eth, history_eth = train_lstm(
    X_train_seq_eth,
    y_train_seq_eth,
    X_val_seq_eth,
    y_val_seq_eth,
    params=formatted_params,
)

Epoch 1/300
52/52 ━━━━━━━━━━━━━━━━━━━━ 6s 28ms/step - loss: 0.0164 - mae: 0.0978 - val_loss: 0.0064 - val_mae: 0.0666
Epoch 2/300
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.0080 - mae: 0.0656 - val_loss: 0.0033 - val_mae: 0.0483
Epoch 3/300
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 0.0056 - mae: 0.0546 - val_loss: 0.0015 - val_mae: 0.0292
Epoch 4/300
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 0.0046 - mae: 0.0489 - val_loss: 0.0017 - val_mae: 0.0337
Epoch 5/300
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 0.0044 - mae: 0.0465 - val_loss: 0.0017 - val_mae: 0.0332
Epoch 6/300
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 0.0042 - mae: 0.0455 - val_loss: 0.0015 - val_mae: 0.0316
Epoch 7/300
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 0.0040 - mae: 0.0439 - val_loss: 0.0016 - val_mae: 0.0307
Epoch 8/300
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - loss: 0.0041 - mae: 0.0450 - val_loss: 0.0014 - val_mae: 0.0291
Epoch 9/300
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - lo

In [35]:
# save weights
model_eth.save_weights(
    config.DATA_DIR
    / "weights"
    / "log_price_change"
    / "log_price_change_weights_eth.weights.h5"
)

#  generate predictions
y_pred_eth = model_eth.predict(X_test_seq_eth).flatten()

# create new df and save it to csv
predictions_eth = pd.DataFrame(
    {
        "date": y_test_eth.index[30:],
        "predicted_log_price_change": y_pred_eth,
    }
)


predictions_eth.to_csv(
    config.DATA_DIR / "predictions" / "log_price_change_predictions_eth.csv",
    index=False,
)

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
